In [5]:
# import
import csv
import math
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from pystruct.models import ChainCRF
from pystruct.learners import FrankWolfeSSVM

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
                                              ExpSineSquared, DotProduct,
                                              ConstantKernel, WhiteKernel)

%matplotlib inline

random.seed
np.set_printoptions(threshold=np.nan)

<bound method Random.seed of <random.Random object at 0x10302a618>>

In [50]:
kernel = 1.4 * RationalQuadratic(length_scale=0.08, alpha=2.1) + 1.0*ExpSineSquared(length_scale = 50, periodicity=1)
# for floor in range(0,5):
#     for building in range(0,3):
for floor in range(1,2):
    for building in range(0,1):
        
        ## Count the number of data points in building id & floor id
        data_num = 0
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num += 1
        print(data_num)
        ## if there are no data, continue to next floor 
        if (data_num == 0):
            continue
            
        ## Load data points in
        wifi_loc_time = np.zeros(shape = (data_num, 524))
        i=-1
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i = i+1
                if (i > data_num):
                    break
                # wifi
                wifi_loc_time[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time[i-1][-1] = np.array(row[-1])
        
        ## Sort by time stamp
        wifi_loc_time = wifi_loc_time[wifi_loc_time[:,-1].argsort()]
        
        ## List all users
        user_list = np.unique(wifi_loc_time[:, 522])
        user_num = len(user_list)
        print("On floor ", floor, " in building ", building, "there are ", user_num, " users.")
        
        ## Make grids
        grid_slice = 20
        grids = 2
        kmeans = KMeans(n_clusters = grids, random_state=0).fit(wifi_loc_time[:, 520:522])
        centers = kmeans.cluster_centers_
        dx = centers[0,0] - centers[1,0]
        dy = centers[0,1] - centers[1,1]
        R = math.sqrt(pow(centers[0,0] - centers[1,0], 2) + pow(centers[0,1] - centers[1,1], 2))
        cos = dx / R
        sin = - dy / R
        rotate = np.array([[cos, -sin],[sin, cos]])
        
        coordinates = np.zeros(shape=(len(wifi_loc_time[:,522]),2))
        
        for i in range(len(wifi_loc_time[:,522])):
            coordinates[i] = np.matmul(rotate, wifi_loc_time[i,520:522])
        
        
                                    ## max boundary , grid size, min boundary
        longitude_list = np.array([max(coordinates[:,0]), (max(coordinates[:,0])-min(coordinates[:,0]))/grid_slice\
                                   , min(coordinates[:,0])])
        latitude_list = np.array([max(coordinates[:,1]), (max(coordinates[:,1])-min(coordinates[:,1]))/grid_slice\
                                   , min(coordinates[:,1])])
        
        ## state tag = [grid_slice x grid_slice]
        state_tag = np.zeros(shape=(grid_slice, grid_slice), dtype=int)
        ## initialize to -1
        for i in range(grid_slice):
            for j in range(grid_slice):
                state_tag[i][j] = -1
        
        ## Record each user's number of location logs
        user_walking_num = np.zeros(shape=(user_num), dtype=int)
        for i in range(len(wifi_loc_time[:,522])):
            for user_index in range(user_num):
                if (wifi_loc_time[i, 522] == user_list[user_index]):
                    user_walking_num[user_index] += 1
        
        ## set the sequence length of observed walking tracks
        seq_window = 9
        total_batch_num = 0
        
        ## compute all users' walking sequences and add them as number of batches
        for num in user_walking_num:
            total_batch_num += math.floor(num/seq_window)
        print("total_batch_num: ",total_batch_num)
        
        batch_wifi = np.zeros((total_batch_num * seq_window, 520))
        batch_loc = np.zeros((total_batch_num * seq_window), dtype = int)
        
        ## tracking parameters
        state_count = 0
        ## cumulative graduation [# of batch of user1] + [# of batch of user2] [...]
        batch_num = 0
        iii = 0
        state_map_grid = {}
        
        for k in range(len(user_list)):
            print("User",k, " walking ", user_walking_num[k])
            
            ## [longitude, latitude, wifi]
            user_fp = np.zeros((user_walking_num[k], 2+520))
            batch_num += math.floor(user_walking_num[k]/seq_window)
            
            ## pick out the corresponding user's data
            user_i = 0
            for i in range(len(wifi_loc_time[:,522])):
                if (wifi_loc_time[i, 522] == user_list[k]):
                    user_fp[user_i, :2] = np.array(coordinates[i])
                    user_fp[user_i, 2:] = np.array(wifi_loc_time[i, :520])
                    user_i += 1
        
            for s in range(math.floor(user_walking_num[k]/seq_window) * seq_window):
                ## put state id
                i = int(math.ceil((user_fp[s][0] - longitude_list[2])/longitude_list[1]) - 1)
                j = int(math.ceil((user_fp[s][1] - latitude_list[2])/latitude_list[1]) - 1)
                if (i == -1):
                    i = 0
                elif (i >= grid_slice): 
                    i = grid_slice-1
                if (j == -1):
                    j = 0
                elif (j >= grid_slice): 
                    j = grid_slice-1
                if (state_tag[i][j] == -1):
                    ## apply GPs
                    gp = GaussianProcessRegressor(kernel=kernel,
                                          alpha=0.5)
                    grid_wifi_data = user_fp[s][2:].copy()
                    grid_loc_data = np.array([user_fp[s][0] - (longitude_list[1]*(float(i)+0.5) + longitude_list[2])\
                                              , user_fp[s][1] - (latitude_list[1]*(float(j)+0.5) + latitude_list[2])])
                    
                    state_tag[i][j] = state_count
                    state_map_grid[state_count] = [i,j, gp, grid_wifi_data, grid_loc_data]
                    state_count += 1
                else:
                    state_map_grid[state_tag[i][j]][3] = np.vstack((state_map_grid[state_tag[i][j]][3], user_fp[s][2:].copy()))
                    state_map_grid[state_tag[i][j]][4] = np.vstack((state_map_grid[state_tag[i][j]][4], \
                               np.array([user_fp[s][0] - (longitude_list[1]*(float(i)+0.5) + longitude_list[2])\
                                        , user_fp[s][1] - (latitude_list[1]*(float(j)+0.5) + latitude_list[2])])))
                    
                batch_wifi[iii] = user_fp[s][2:].copy()
                batch_loc[iii] = state_tag[i][j]
                iii += 1

        print("Grid is ", grid_slice, "x", grid_slice, " ; the # of states is ", state_count)
        for s in range(state_count):
            print("State ", s, "; data points: ", state_map_grid[s][4].shape)
            state_map_grid[s][2] = state_map_grid[s][2].fit(state_map_grid[s][3], state_map_grid[s][4])
                    
        # Testing
        # extract data
        print("Validation...")
        data_num_v = 0
        with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
            spamreader_v = csv.reader(v_csvfile, delimiter=',')
            for row in spamreader_v:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num_v += 1

        print(data_num_v)
        if (data_num_v == 0):
            continue
        i = -1
        wifi_loc_time_v = np.zeros(shape = (data_num_v, 524))
        
        with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
            spamreader_v = csv.reader(v_csvfile, delimiter=',')
            for row in spamreader_v:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i += 1
                #if (i > data_num):
                #    break
                # wifi
                wifi_loc_time_v[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time_v[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time_v[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time_v[i-1][-1] = np.array(row[-1])
                
        wifi_loc_time_v = wifi_loc_time_v[wifi_loc_time_v[:,-1].argsort()]  
                
        batch_num = int(data_num_v/seq_window)
        print("Validating data ", batch_num)
        batch_wifi_v = np.zeros(((batch_num + 1) * seq_window, 520))
        batch_loc_v = np.zeros(((batch_num + 1) * seq_window), dtype = int)

        iter_window = 0
        iter_batch = 1
        
        map_distance = 0
        adjust_grid_num = 0
                               
        coordinates_v = np.zeros(shape=(len(wifi_loc_time_v[:,522]),2))
        
        for i in range(len(wifi_loc_time_v[:,522])):
            coordinates_v[i] = np.matmul(rotate, wifi_loc_time_v[i,520:522])
            
        iii = seq_window

        for s in range(batch_num*seq_window):
            
            i = int(math.ceil((coordinates_v[s][0]-longitude_list[2])/longitude_list[1]) - 1)
            j = int(math.ceil((coordinates_v[s][1]-latitude_list[2])/latitude_list[1]) - 1)
            if (i < -1):
                i = 0
            elif (i >= grid_slice): 
                i = grid_slice-1
            if (j < 0):
                j = 0
            elif (j >= grid_slice): 
                j = grid_slice-1
            if (state_tag[i][j] == -1):
                adjust_grid_num+=1
                min_dist = np.zeros(shape=(state_count, 1))
                for x in range(state_count):
                    min_dist[x,0] = math.sqrt(pow((float(state_map_grid[x][0])+0.5)*longitude_list[1]+longitude_list[2] - coordinates_v[s][0],2)+\
                              pow((float(state_map_grid[x][1])+0.5)*latitude_list[1]+latitude_list[2] - coordinates_v[s][1],2))
                map_distance += np.min(min_dist)
                state_tag[i][j] = np.argmin(min_dist)
            
            batch_wifi_v[iii] = wifi_loc_time_v[s][:520]
            batch_loc_v[iii] = state_tag[i][j]
            iii += 1
             
        print("Grids need adjustment: ", adjust_grid_num)
        print("Base map distance is ", map_distance/data_num_v)
                
        neigh = KNeighborsClassifier(n_neighbors=1)
        neigh.fit(batch_wifi, batch_loc)
        pred_batch_v = neigh.predict(batch_wifi_v)

        center_longitude_pred = 0
        center_latitude_pred = 0
        center_longitude_gt = 0
        center_latitude_gt = 0
        
        real_longitude = 0
        real_latitude = 0

        iter_test = 0
        
        gp_total_error = 0
        gp_window = np.zeros((seq_window))
        
        error_list = []
        ans_list_lo = []
        ans_list_la = []
        center_lo = []
        center_la = []
                                 
        inverse = np.linalg.inv(rotate)
        
        ## State prediction evaluation
        accuracy_state_tag = 0
        rmse_state_tag = 0
        rmse_grid_center = 0
        
        ## Ground-truth RMSE
        rmse_gt = 0
        error_list_gt = []
        total_error_gt = 0
        window_gt = np.zeros((seq_window))
        
        for i in range(1,batch_num+1):
            for j in range(seq_window):
                
                ## Offset produced by Gaussian Process
                ans = state_map_grid[pred_batch_v[i*seq_window + j]][2].predict(batch_wifi_v[i*seq_window + j].reshape(1,520))
                
                ## Offset produced by Ground-truth Gaussian Process
                ans_gt = state_map_grid[batch_loc_v[i*seq_window + j]][2].predict(batch_wifi_v[i*seq_window + j].reshape(1,520))
                
                center_longitude_pred = (float(state_map_grid[pred_batch_v[i*seq_window + j]][0])+0.5)*longitude_list[1]
                center_latitude_pred = (float(state_map_grid[pred_batch_v[i*seq_window + j]][1])+0.5)*latitude_list[1]
                center_longitude_gt = (float(state_map_grid[batch_loc_v[i*seq_window + j]][0])+0.5)*longitude_list[1]
                center_latitude_gt = (float(state_map_grid[batch_loc_v[i*seq_window + j]][1])+0.5)*latitude_list[1]
                
                real_longitude = wifi_loc_time_v[iter_test][520]
                real_latitude = wifi_loc_time_v[iter_test][521]
                
                ## State tagging part
                cen = np.matmul(inverse, np.array([center_longitude_pred+longitude_list[2],\
                                                   center_latitude_pred+latitude_list[2]]))
                center_lo.append(cen[0])
                center_la.append(cen[1])
                rmse_state_tag += math.sqrt(pow(cen[0] - real_longitude,2)\
                                         +pow(cen[1] - real_latitude,2))
                if (pred_batch_v[i*seq_window + j] == batch_loc_v[i*seq_window + j]):
                    accuracy_state_tag += 1
                ## RMSE between centers
                rmse_grid_center += math.sqrt(pow(center_longitude_pred - center_longitude_gt,2)\
                                              +pow(center_latitude_pred - center_latitude_gt,2))
                
                ## Add base and offset
                ans[0][0] += center_longitude_pred + longitude_list[2]
                ans[0][1] += center_latitude_pred + latitude_list[2]
                
                ans = np.matmul(inverse, np.array([ans[0][0], ans[0][1]]))

                ## RMSE between Final prediction result & real location
                error_list.append(math.sqrt(pow(ans[0] - real_longitude,2)+pow(ans[1] - real_latitude,2)))
                gp_total_error += error_list[-1]
                gp_window[j] += error_list[-1]
                ## store ans
                ans_list_lo.append(ans[0])
                ans_list_la.append(ans[1])
                
                ## Ground_truth
                ## Add base and offset
                ans_gt[0][0] += center_longitude_gt + longitude_list[2]
                ans_gt[0][1] += center_latitude_gt + latitude_list[2]
                
                ans_gt = np.matmul(inverse, np.array([ans_gt[0][0], ans_gt[0][1]]))

                ## RMSE between Final prediction result & real location
                error_list_gt.append(math.sqrt(pow(ans_gt[0] - real_longitude,2)+pow(ans_gt[1] - real_latitude,2)))
                total_error_gt += error_list_gt[-1]
                window_gt[j] += error_list_gt[-1]
                
                iter_test+=1
                

        print("\n\nState tagging error\n RMSE w.r.t. real location: ", rmse_state_tag/(batch_num*seq_window)\
             ,"\n RMSE between gt center and predicted center: ", rmse_grid_center/(batch_num*seq_window)\
             ,"\n Accuracy of state tagging: ", accuracy_state_tag/(batch_num*seq_window))
        
        print("\nGP part:\n RMSE: ", gp_total_error/(batch_num*seq_window)\
              ,"\n Windows:")
        for x in range(seq_window):
            print(" ", x, gp_window[x]/batch_num)
            
        print("\nGround-truth part:\n RMSE: ", total_error_gt/(batch_num*seq_window)\
              ,"\n Windows:")
        for x in range(seq_window):
            print(" ", x, window_gt[x]/batch_num)
        print("Finish\n")

1356
On floor  1  in building  0 there are  2  users.
total_batch_num:  150
User 0  walking  757
User 1  walking  599
Grid is  20 x 20  ; the # of states is  48
State  0 ; data points:  (70, 2)
State  1 ; data points:  (30, 2)
State  2 ; data points:  (29, 2)
State  3 ; data points:  (78, 2)
State  4 ; data points:  (70, 2)
State  5 ; data points:  (29, 2)
State  6 ; data points:  (24, 2)
State  7 ; data points:  (20, 2)
State  8 ; data points:  (39, 2)
State  9 ; data points:  (29, 2)
State  10 ; data points:  (19, 2)
State  11 ; data points:  (20, 2)
State  12 ; data points:  (18, 2)
State  13 ; data points:  (19, 2)
State  14 ; data points:  (20, 2)
State  15 ; data points:  (40, 2)
State  16 ; data points:  (40, 2)
State  17 ; data points:  (19, 2)
State  18 ; data points:  (10, 2)
State  19 ; data points:  (20, 2)
State  20 ; data points:  (46, 2)
State  21 ; data points:  (20, 2)
State  22 ; data points:  (20, 2)
State  23 ; data points:  (20, 2)
State  24 ; data points:  (30, 2)

In [31]:
batch_loc_v

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0, 20, 28, 26, 25, 23, 23, 21, 21,
       43, 20, 17,  8, 45, 45,  3,  5,  6,  8, 20, 44,  7,  5, 45,  8,  0,
        0, 17,  9,  9, 10, 11, 12, 12, 12, 12, 14, 20, 44, 43, 39, 31, 31,
       31, 33, 33, 34, 34, 36, 36, 36, 39, 43, 43, 44, 44, 44, 39, 45, 45,
       45,  1,  1,  1, 45, 45, 45, 45,  8,  1, 44, 17, 39, 31, 32, 35, 15,
        6, 28, 27, 38, 15, 37, 25, 26,  7, 15, 22, 12,  3, 29, 11, 39, 10,
        9, 31, 22, 23, 17, 35,  1, 30, 10, 37, 12,  2, 39, 28,  5, 24,  6,
       31, 35, 26,  6, 21, 26, 33, 43,  6, 28, 44, 17,  2,  4,  6, 36,  6,
       37, 37, 44, 16, 11, 14, 15,  8,  2,  6, 43, 21, 25,  6, 28,  6, 25,
       31, 31, 33, 35, 38, 37, 36, 37, 20,  0,  4,  6,  6,  5, 13, 12,  9,
       18])

In [32]:
pred_batch_v

array([36, 36, 36, 36, 36, 36, 36, 36, 36, 19, 27, 27, 24, 21, 26, 24, 25,
       42, 19,  8, 10,  0, 45,  5,  5,  6,  0, 19, 44,  5,  5, 45,  0, 10,
        0, 10,  0, 10, 10, 12, 14, 10, 12, 12, 10, 20, 44, 42, 26, 26, 21,
       25, 33, 35, 32, 35, 32, 35, 36, 25, 42, 44, 19, 42, 44, 24,  0, 18,
        9, 45,  7, 45,  0,  0,  8,  8,  0, 45, 44, 19, 21, 31, 33, 33, 10,
       28, 27, 27, 46, 13, 35, 23, 22,  5, 10, 21, 11,  8, 25,  9, 21,  8,
        8, 21, 22, 23,  0, 32,  0, 24, 12, 21,  0,  8, 42, 22,  5, 24,  5,
       24, 33, 23,  5, 21, 26, 31, 43,  5, 25, 42, 17, 19, 19,  7, 36,  5,
       46, 31, 44,  0,  9, 13, 13,  8,  0,  0, 44, 30, 29, 25, 24, 27, 25,
       27, 21, 33, 24, 21, 21, 36, 36, 42,  8,  4,  6,  6,  5, 12, 12,  8,
       10])